In [30]:
import numpy as np
import sympy
import models
import utilities

# Models
`models.stochastic.StochasticModel` is a class that defines any nonlinear dynamical model influenced by stochastic noise parameters. For convenience, a number of functions in models.parasitism exist to make some simple models, including a test AR1 autoregressive model, `ar1()`; Nicholson-Bailey, `nb()`; Nicholson-Bailey with negative binomial functional response, `nbd()`; and a helper function for creating multi-patch models with constant, symmetric linear dispersal between patches, `get_model(modelstr)`. 

In [10]:
model = models.parasitism.get_model("nbd(2)") # Get a two-patch NBD model.

`models.stochastic.StochasticModel` contains a number of methods for solving for equilibria, deriving the linearized model, etc. For convenience, `models.parasitism.get_model` automatically solves for the equilibrium and linearization, so you can access them through their respective member variables.

### Member variables

In [16]:
model.vars # deterministic variables (host, parasitoid for each patch.)

Matrix([
[H^{(1)}],
[H^{(2)}],
[P^{(1)}],
[P^{(2)}]])

In [18]:
model.noises # stochastic variables

Matrix([
[\epsilon^{(1)}_h],
[\epsilon^{(2)}_h],
[\epsilon^{(1)}_p],
[\epsilon^{(2)}_p]])

In [19]:
model.deterministic # deterministic terms of the equation

Matrix([
[H^{(1)}*\lambda*(-\mu_H + 1)*(P^{(1)}*a/k + 1)**(-k) + H^{(2)}*\lambda*\mu_H*(P^{(2)}*a/k + 1)**(-k)/2],
[H^{(1)}*\lambda*\mu_H*(P^{(1)}*a/k + 1)**(-k)/2 + H^{(2)}*\lambda*(-\mu_H + 1)*(P^{(2)}*a/k + 1)**(-k)],
[H^{(1)}*c*(1 - (P^{(1)}*a/k + 1)**(-k))*(-\mu_P + 1) + H^{(2)}*\mu_P*c*(1 - (P^{(2)}*a/k + 1)**(-k))/2],
[H^{(1)}*\mu_P*c*(1 - (P^{(1)}*a/k + 1)**(-k))/2 + H^{(2)}*c*(1 - (P^{(2)}*a/k + 1)**(-k))*(-\mu_P + 1)]])

In [20]:
model.stochastic # stochastic terms of the equation

Matrix([
[H^{(1)}*\lambda*(-\mu_H + 1)*(P^{(1)}*a/k + 1)**(-k)*exp(\epsilon^{(1)}_h) + H^{(2)}*\lambda*\mu_H*(P^{(2)}*a/k + 1)**(-k)*exp(\epsilon^{(2)}_h)/2],
[H^{(1)}*\lambda*\mu_H*(P^{(1)}*a/k + 1)**(-k)*exp(\epsilon^{(1)}_h)/2 + H^{(2)}*\lambda*(-\mu_H + 1)*(P^{(2)}*a/k + 1)**(-k)*exp(\epsilon^{(2)}_h)],
[H^{(1)}*c*(1 - (P^{(1)}*a/k + 1)**(-k))*(-\mu_P + 1)*exp(\epsilon^{(1)}_p) + H^{(2)}*\mu_P*c*(1 - (P^{(2)}*a/k + 1)**(-k))*exp(\epsilon^{(2)}_p)/2],
[H^{(1)}*\mu_P*c*(1 - (P^{(1)}*a/k + 1)**(-k))*exp(\epsilon^{(1)}_p)/2 + H^{(2)}*c*(1 - (P^{(2)}*a/k + 1)**(-k))*(-\mu_P + 1)*exp(\epsilon^{(2)}_p)]])

In [21]:
model.equilibrium

{H^{(2)}: -k*(a*(\lambda*k**k)**(1/k))**k/(-a*a**k*c*k**k + a*c*(a*(\lambda*k**k)**(1/k))**k) + (\lambda*k**k)**(1/k)*(a*(\lambda*k**k)**(1/k))**k/(-a*a**k*c*k**k + a*c*(a*(\lambda*k**k)**(1/k))**k),
 P^{(2)}: -k/a + (\lambda*k**k)**(1/k)/a,
 P^{(1)}: -k/a + (\lambda*k**k)**(1/k)/a,
 H^{(1)}: -k*(a*(\lambda*k**k)**(1/k))**k/(-a*a**k*c*k**k + a*c*(a*(\lambda*k**k)**(1/k))**k) + (\lambda*k**k)**(1/k)*(a*(\lambda*k**k)**(1/k))**k/(-a*a**k*c*k**k + a*c*(a*(\lambda*k**k)**(1/k))**k)}

In [23]:
model.A # Deterministic Jacobian evaluated at the equilibrium point.

Matrix([
[\lambda*(-\mu_H + 1)*(a*(-k/a + (\lambda*k**k)**(1/k)/a)/k + 1)**(-k),      \lambda*\mu_H*(a*(-k/a + (\lambda*k**k)**(1/k)/a)/k + 1)**(-k)/2, -\lambda*a*(-\mu_H + 1)*(a*(-k/a + (\lambda*k**k)**(1/k)/a)/k + 1)**(-k)*(-k*(a*(\lambda*k**k)**(1/k))**k/(-a*a**k*c*k**k + a*c*(a*(\lambda*k**k)**(1/k))**k) + (\lambda*k**k)**(1/k)*(a*(\lambda*k**k)**(1/k))**k/(-a*a**k*c*k**k + a*c*(a*(\lambda*k**k)**(1/k))**k))/(a*(-k/a + (\lambda*k**k)**(1/k)/a)/k + 1),    -\lambda*\mu_H*a*(a*(-k/a + (\lambda*k**k)**(1/k)/a)/k + 1)**(-k)*(-k*(a*(\lambda*k**k)**(1/k))**k/(-a*a**k*c*k**k + a*c*(a*(\lambda*k**k)**(1/k))**k) + (\lambda*k**k)**(1/k)*(a*(\lambda*k**k)**(1/k))**k/(-a*a**k*c*k**k + a*c*(a*(\lambda*k**k)**(1/k))**k))/(2*(a*(-k/a + (\lambda*k**k)**(1/k)/a)/k + 1))],
[     \lambda*\mu_H*(a*(-k/a + (\lambda*k**k)**(1/k)/a)/k + 1)**(-k)/2, \lambda*(-\mu_H + 1)*(a*(-k/a + (\lambda*k**k)**(1/k)/a)/k + 1)**(-k),    -\lambda*\mu_H*a*(a*(-k/a + (\lambda*k**k)**(1/k)/a)/k + 1)**(-k)*(-k*(a*(\lambda*k**

In [24]:
model.B # Stochastic Jacobian evaluated at the equilibrium point.

Matrix([
[\lambda*(-\mu_H + 1)*(a*(-k/a + (\lambda*k**k)**(1/k)/a)/k + 1)**(-k)*(-k*(a*(\lambda*k**k)**(1/k))**k/(-a*a**k*c*k**k + a*c*(a*(\lambda*k**k)**(1/k))**k) + (\lambda*k**k)**(1/k)*(a*(\lambda*k**k)**(1/k))**k/(-a*a**k*c*k**k + a*c*(a*(\lambda*k**k)**(1/k))**k)),      \lambda*\mu_H*(a*(-k/a + (\lambda*k**k)**(1/k)/a)/k + 1)**(-k)*(-k*(a*(\lambda*k**k)**(1/k))**k/(-a*a**k*c*k**k + a*c*(a*(\lambda*k**k)**(1/k))**k) + (\lambda*k**k)**(1/k)*(a*(\lambda*k**k)**(1/k))**k/(-a*a**k*c*k**k + a*c*(a*(\lambda*k**k)**(1/k))**k))/2,                                                                                                                                                                                                                                                                    0,                                                                                                                                                                                                             

### Methods
`StochasticModel` contains a few methods for calculating covariance, eigenvalues, the spectral matrix, and simulation results.

In [34]:
params = dict(r=2.0, a=1.0, c=1.0, k=0.5, mh=0.25, mp=0.25)
noiseparams = dict(SpSh=1, Chh=0.5, Cpp=0.5)

# Simple helper for making noise covariance matrices.
noise = utilities.noise_cov(noiseparams) 

# Converts text parameter names to sympy symbols.
sym_params = models.parasitism.sym_params(params) 

print noise
print sym_params

[[ 1.   0.5  0.   0. ]
 [ 0.5  1.   0.   0. ]
 [ 0.   0.   1.   0.5]
 [ 0.   0.   0.5  1. ]]
{\mu_P: 0.25, a: 1.0, \mu_H: 0.25, c: 1.0, k: 0.5, \lambda: 2.0}


#### Eigenvalues

In [48]:
model.calculate_eigenvalues(sym_params)

(array([ 0.6015625+0.46080507j,  0.6015625-0.46080507j,
         0.4296875+0.32914648j,  0.4296875-0.32914648j]),
 array([[ 0.54772256 +0.00000000e+00j,  0.54772256 -0.00000000e+00j,
         -0.54772256 +0.00000000e+00j, -0.54772256 -0.00000000e+00j],
        [ 0.54772256 +6.38378239e-16j,  0.54772256 -6.38378239e-16j,
          0.54772256 +8.60422844e-16j,  0.54772256 -8.60422844e-16j],
        [ 0.22821773 -3.84599359e-01j,  0.22821773 +3.84599359e-01j,
         -0.22821773 +3.84599359e-01j, -0.22821773 -3.84599359e-01j],
        [ 0.22821773 -3.84599359e-01j,  0.22821773 +3.84599359e-01j,
          0.22821773 -3.84599359e-01j,  0.22821773 +3.84599359e-01j]]))

##### Covariance
There are two methods for calculating the model's covariance. One integrates the covariance from the analytic spectral matrix. The other is completely analytic and much faster.

In [46]:
import timeit

start_time = timeit.default_timer()
cov1 = model.integrate_covariance_from_analytic_spectrum(sym_params, noise, nfreqs=4096)
elapsed_1 = timeit.default_timer() - start_time

start_time = timeit.default_timer()
cov2 = model.calculate_covariance(sym_params, noise)
elapsed_2 = timeit.default_timer() - start_time

print '%fs' % elapsed_1
print cov1
print
print '%fs' % elapsed_2
print cov2

0.287349s
[[ 14.60445527  11.85520339   7.93934652   6.93348083]
 [ 11.85520339  14.60445527   6.93348083   7.93934652]
 [  7.93934652   6.93348083   5.94084378   5.12252853]
 [  6.93348083   7.93934652   5.12252853   5.94084378]]

0.000717s
[[ 14.60764234  11.85786537   4.09783558   3.65068681]
 [ 11.85786537  14.60764234   3.65068681   4.09783558]
 [  4.09783558   3.65068681   5.94208431   5.12363945]
 [  3.65068681   4.09783558   5.12363945   5.94208431]]


#### Spectral matrix

In [52]:
# Adjust v to change of input frequency at which to evaluate the spectral matrix (0 to 0.5)
model.calculate_spectrum(sym_params, noise, v=0)

array([[ 23.97826365+0.j,  17.98392472+0.j,  11.17457108+0.j,
          9.34842061+0.j],
       [ 17.98392472+0.j,  23.97826365+0.j,   9.34842061+0.j,
         11.17457108+0.j],
       [ 11.17457108+0.j,   9.34842061+0.j,   7.95053105+0.j,
          6.70874873+0.j],
       [  9.34842061+0.j,  11.17457108+0.j,   6.70874873+0.j,
          7.95053105+0.j]])

#### Simulation
See `test_cospectrum.py` and `test_covariance.py` for some examples of how to simulate population trajectories.